In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/philippe/MolGenDocking


/home/philippe/miniconda3/envs/MolGen/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import numpy as np
import json
import MDAnalysis as mda
import nglview as nv

In [28]:
PATH = 'data/SIU'

pockets = json.load(open(os.path.join(PATH, "target_info.json")))
len(pockets.keys())

34

In [29]:
i=0

In [55]:
def get_selection(size, center, sim):
    dx, dy, dz = size
    x, y, z = center

    # Compute min/max box edges
    xmin, xmax = x - dx/2, x + dx/2
    ymin, ymax = y - dy/2, y + dy/2
    zmin, zmax = z - dz/2, z + dz/2

    # Get atoms
    all_atoms = sim.select_atoms("all")
    coords = all_atoms.positions

    # Create boolean mask for atoms inside the box
    mask = (
        (coords[:, 0] > xmin) & (coords[:, 0] < xmax) &
        (coords[:, 1] > ymin) & (coords[:, 1] < ymax) &
        (coords[:, 2] > zmin) & (coords[:, 2] < zmax)
    )

    # Apply mask
    pocket_atoms = all_atoms[mask]
    pocket_residues = pocket_atoms.residues

    # Use segid instead of chainID
    pocket_selection = " or ".join([f"resi {res.resid}" for res in pocket_residues])
    return pocket_selection


In [62]:
pdb_id = list(pockets.keys())[i]
i+=1
print(pdb_id)
u = mda.Universe(os.path.join(PATH, "pdb_files", f"{pdb_id}.pdb"))

center = pockets[pdb_id]['center']
radius = np.mean(size)/10

pocket_selection = get_selection(
    size=np.array(pockets[pdb_id]['size']),
    center=np.array(center),
    sim=u
)

view = nv.show_mdanalysis(u)

view.add_representation("surface", selection=pocket_selection, colorScheme="hydrophobicity")
view

2wm3


NGLWidget()

In [50]:
center

[52.434799999999996, 47.922599999999996, 2.72916]

In [51]:
x

52.434799999999996